## Long Sentence Sequence

문장이 매우 긴 경우,

ㅁㅁㅁ ㅁㅁㅁㅁ ㅁㅁㅁㅁ ㅁㅁ ㅁㅁㅁ.

1. ■ ■ ■ ■ ㅁㅁㅁ ㅁㅁㅁㅁ ㅁㅁ ㅁㅁㅁ.
2. ㅁ■ ■ ■ ■ ㅁㅁ ㅁㅁㅁㅁ ㅁㅁ ㅁㅁㅁ.
3. ㅁㅁ■ ■ ■ ■ ㅁ ㅁㅁㅁㅁ ㅁㅁ ㅁㅁㅁ.

ㄴ> 이런 식으로 한 칸씩 옆으로 밀면서 학습함.

x : 밀기 전 값 / y : 밀고 난 후 값

In [1]:
import numpy as np
import torch
from torch import optim
import torch.nn.functional as F 

In [2]:
sentence = "I am so happy to study RNN. There are so many things to study. It's amazing."

x_data = []
y_data = []
sequence_length = 15
hidden_size = input_size = sequence_length

In [3]:
# char_dic 정의
char_dic = {char: i for i, char in enumerate(set(sentence))}
dic_size = len(char_dic)

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1:i + sequence_length + 1]

    print(i, x_str, '->', y_str)
    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])

x_one_hot = [np.eye(dic_size)[x] for x in x_data]

x = torch.FloatTensor(x_one_hot)
y = torch.LongTensor(y_data)

0 I am so happy t ->  am so happy to
1  am so happy to -> am so happy to 
2 am so happy to  -> m so happy to s
3 m so happy to s ->  so happy to st
4  so happy to st -> so happy to stu
5 so happy to stu -> o happy to stud
6 o happy to stud ->  happy to study
7  happy to study -> happy to study 
8 happy to study  -> appy to study R
9 appy to study R -> ppy to study RN
10 ppy to study RN -> py to study RNN
11 py to study RNN -> y to study RNN.
12 y to study RNN. ->  to study RNN. 
13  to study RNN.  -> to study RNN. T
14 to study RNN. T -> o study RNN. Th
15 o study RNN. Th ->  study RNN. The
16  study RNN. The -> study RNN. Ther
17 study RNN. Ther -> tudy RNN. There
18 tudy RNN. There -> udy RNN. There 
19 udy RNN. There  -> dy RNN. There a
20 dy RNN. There a -> y RNN. There ar
21 y RNN. There ar ->  RNN. There are
22  RNN. There are -> RNN. There are 
23 RNN. There are  -> NN. There are s
24 NN. There are s -> N. There are so
25 N. There are so -> . There are so 
26 . There are so  -> 

C:\Users\kuroc\AppData\Local\Temp\ipykernel_16832\1812636438.py:15: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:264.)
  x = torch.FloatTensor(x_one_hot)


In [4]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, dic_size, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        # RNN 레이어의 출력 크기를 이해하고, 세 번째 차원에서 마지막 타임 스텝의 출력을 선택
        x = x[:, -1, :]  # x의 크기는 (batch_size, sequence_length, hidden_size)
        x = self.fc(x)
        return x


In [5]:
net = Net(dic_size, hidden_size, 2)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

In [9]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(x)

    # softmax 적용
    outputs_softmax = F.softmax(outputs, dim=1)

    # outputs에 softmax를 적용한 후 손실을 계산
    loss = criterion(outputs_softmax.view(-1, dic_size), y.view(-1)[:outputs_softmax.view(-1, dic_size).shape[0]])

    loss.backward()
    optimizer.step()

    result = outputs.argmax(dim=1)
    predict_str = ""
    for j, r in enumerate(result):
        if torch.numel(r) == 1:
            # 스칼라인 경우 처리
            predict_str += sentence[int(r)]
        elif j == 0:
            predict_str += ''.join([ sentence[t] for t in r])
            print(predict_str)
        else:
            predict_str += char_set[int(r[-1])]
    print(predict_str)

IIIIpyIo   IIhy  IpyIo    IhyI IpyIo   IIhyI IpyIo   yIhyIohp
IIIIpyIo   IIhy  IpyIo   IIhyI IpyIo   IIhyI IpyIo   yIhyIohp
IIIIpyIo   IIhy  IpyIo   IIhyI IpyIo   IIhyI IpyIo   yIhyIohp
IIIIpyIo   IIhy  IpyIo   IIhyI IpyIo   IIhyI IpyIo   yIhyIohp
IIIIpyIo   IIhy  IpyIo   IIhyI IpyIo   IIhyI IpyIo   yIhyIIhp
IIIIpyIo   IIhy  IpyIo   IIhyI IpyIo   IIhyI IpyIo   yIhyIIhp
IIIIpyIo   IIhy  IpyIo   IIhyI IpyIo   IIhyI IpyIo   yIhyIIhp
IIIIpyIo   IIhy  IpyIo   IIhyI IpyIo   IIhyI IpyIo   yIhyIIhp
IIIIpyIo   IIhy  IpyIo   IIhyI IpyIo   IIhyI IpyIo   yIhyIIhp
IIIIpyIo   IIhy  IpyIo   IIhyI IpyIo   IIhyI IpyIo   yIhyIIhp
IIIIpyIo   IIhy  IpyIo   IIhyI IpyIo   IIhyI IpyIo   yIhyIIhp
IIIIpyIo   IIhy  IpyIo    IhyI IpyIo   IIhyI IpyIo   yIhyIIhp
IIIIpyIo   IIhy  IpyIo    IhyI IpyIo   IIhyI IpyIo   yIhyIIhp
IIIIpyIo   IIhy  IpyIo    IhyI IpyIo   IIhyI IpyIo   yIhyIIhp
IIIIpyIo   IIhy  IpyIo    IhyI IpyIo   IIhyI IpyIo   yIhyIIhp
I IIpyIo   IIhy  IpyIo    IhyI IpyIo   IIhyI IpyIo   yIhyIohp
I IIpyIo